In [29]:
import socket
#import send
#import Path
import cv2 as cv
import numpy as np
import math
from numpy.lib.function_base import disp

# ip = "192.168.43.9"
ip = '192.168.43.9'
'''This ip is your server machine's ip '''
port = 12362

def takepic():
    orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video2 2


def send_common(text):

    c = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    c.connect((ip, port))

    c.send(bytes(text, encoding="utf-8"))
    rep = c.recv(1000)
    print(str(rep,encoding="utf-8"))
    c.close()
    return str(rep, encoding="utf-8")


def get_data_frame(text):
    c = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    c.connect((ip, port))
    c.send(bytes(text, encoding="utf-8"))
    rep = c.recv(1000)
    c.close()

    message = eval(rep)
    Location = message['Location']
    Barrier = message['Barrier']
    Target = message['Target']
    Start = message['Start']
    End = message['End']

    print('Location:', Location,'Barrier:', Barrier,'Target:', Target,'Start:', Start, 'End:',End)
    return Location, Barrier, Target, Start, End


In [30]:
def sendok():
    print('OK!!!!!!!')

In [31]:
import binascii
import serial
import os
os.system('sh ./stop_sys_ttyPS0.sh')
def run_action(cmd):
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    cnt_err = 0
    while 1:
        test_read = ser.read()
        print('test_read', test_read)
        cnt_err += 1
        if test_read== b'\xa3' or cnt_err == 50:
            break
    
    if cnt_err == 50:
        print('can not get REQ')
    else:
        print('read REQ finished!')
        ser.write(cmd2data(cmd))
        print('send action ok!')
    ser.close()
    
def crc_calculate(package):
    crc = 0
    for hex_data in package:

        b2 = hex_data.to_bytes(1, byteorder='little')
        crc = binascii.crc_hqx(b2, crc)

    return [(crc >> 8), (crc & 255)]    # 校验位两位

def cmd2data(cmd):
    cnt=0
    cmd_list=[]
    for i in cmd:
        cnt+=1
        cmd_list+=[ord(i)]
    cmd_list=[0xff,0xff]+[(cnt+5)>>8,(cnt+5)&255]+[0x01,(cnt+1)&255,0x03]+cmd_list
    cmd_list=cmd_list+crc_calculate(cmd_list)
    return cmd_list

def wait_req():
    ser = serial.Serial("/dev/ttyPS0", 9600, timeout=5)
    while 1:
        test_read=ser.read()
        if test_read== b'\xa3' :
            print('read REQ finished!') 
            break

In [32]:
def center():
    ff = 0
    lrmin= 250 #######TODO
    rrmin=250  #######TODO
    mk_min = 10
    rmax = 100   #######TODO
    mrmax = 250
    mrmin = 150
    while(ff==0):
        takepic()
        img = cv.imread('/home/xilinx/jupyter_notebooks/common/data/webcam.jpg')
        [lk,lr,mk,mr,rk,rr] = houfline(img)
        if(mk==0):
            run_action('Back2Run')
            wait_req()
            print('back because mk==0')
            run_action('Stand')
            wait_req()
        elif(abs(mk)<mk_min):
            if(mk>0):
                #left
                run_action('turn001L')
                wait_req()
            elif(mk<0):    
                run_action('turn001R')
                wait_req()
        elif(mr<mrmin):
            run_action('Back2Run')
            #run_action('fastForward03')
            wait_req()
            print('back because too close')
            run_action('Stand')
            wait_req()
        elif(mr>mrmax):
            run_action('Forwalk02')########TODO
            #run_action('fastForward03')
            wait_req()
        elif(lk!=0 and rk!=0):
            if(abs(lr-rr)>rmax):
                if(lr>rr):
                    run_action('Left3move')
                    wait_req()
                elif(lr<rr):
                    run_action('Right3move')
                    wait_req()
            else:
                ff = 1        
        elif(lk==0 or rk == 0):
            if(lk!=0):
                if(lr<lrmin):
                    run_action('Right3move')
                    wait_req()    
            elif(rr!=0):
                if(rr<rrmin):
                     run_action('Left3move')
                     wait_req()    

In [33]:
def obj_recog():
    orig_img_path = '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'
    !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2
    # jupyter调用摄像头的代码
    im = cv.imread(orig_img_path)
    #cv.imshow('t',im)
    #cv.imgshow(im)
    width = im.shape[1]
    height = im.shape[0]
    rate = round(width*0.5)
    rate1 = round(height*0.2)
    half = round(height/2)
    ims = im[rate1:half,0:(width-rate)]
    ims_right = im[(half+1):(height-rate1),0:(width-rate)]
    lower_red = np.array([0, 43, 20]) # 红色范围低阈值
    upper_red = np.array([20, 255, 255]) # 红色范围高阈值
    lower_red2 = np.array([150, 43, 20]) # 红色范围低阈值
    upper_red2 = np.array([200, 255, 255]) # 红色范围高阈值
    hsv_img = cv.cvtColor(ims, cv.COLOR_BGR2HSV)
    mask_red = cv.inRange(hsv_img, lower_red, upper_red)
    mask_red2 = cv.inRange(hsv_img, lower_red2, upper_red2)
    mask = cv.bitwise_or(mask_red,mask_red2)
    # again for right side
    hsv_img_right = cv.cvtColor(ims_right, cv.COLOR_BGR2HSV)
    mask_red_right = cv.inRange(hsv_img_right, lower_red, upper_red)
    mask_red_right2 = cv.inRange(hsv_img_right, lower_red2, upper_red2)
    mask_right = cv.bitwise_or(mask_red_right,mask_red_right2)
    
    #print(mask_red)
    #print(mask_red2)
    #print(mask)
    #print(np.sum(mask_red))
    #print(np.sum(mask_red2))
    #print(np.sum(mask))
    nl = np.sum(mask)
    nr = np.sum(mask_right)
    #print(number)
    number = nl + nr
    
    h1 = height - rate1*2
    w1 = width - rate
    S = h1*w1
    print("nr",nr/255)
    print("nl",nl/255)
    test = nr/255 - nl/255
    print(test )
    print(number/S/255)
    number = (number/S/255)
    
    yu = 2000
    
    if(number < 0.01):
        kind =  0
    else:
        kind = 1
    #if (number>0.01 && number<0.03 && number>0.5):
        #kind = 

    return kind

In [34]:
def houfline(img):
    img = img[:,50:370]
    # cv.imshow("img",img)
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    l_blue = np.array([[100,43,46]])
    h_blue = np.array([[124,255,255]])
    l_green = np.array([[40,100,100]])
    h_green = np.array([[60,200,255]])
    mask_b = cv.inRange(hsv, l_blue, h_blue)
    mask_g = cv.inRange(hsv, l_green, h_green)
    res = cv.bitwise_or(mask_b, mask_g)
    # cv.imshow("test", res)
    # cv.imshow("mask", mask_g)
    list1 = []
    edges = cv.Canny(res,50,150,apertureSize = 3)
    # cv.imshow("mask2", edges)
    #lines = cv.HoughLinesP(mask_g,1,np.pi/180,100,minLineLength=100,maxLineGap=1)
    lines = cv.HoughLinesP(edges,1,np.pi/180,70,minLineLength=10,maxLineGap=100)
    if lines is None:
        return 0,0,0
    for line in lines:
        x1,y1,x2,y2 = line[0]
        #print(x1,y1,x2,y2)
        m = (y1-y2)/(x1-x2)
        p = [0,240]
        a = y1-y2
        b = x2-x1
        c = x1*y2 - y1*x2
        dis = abs(a*p[0]+b*p[1]+c)/math.sqrt(a*a+b*b)
        #print(dis,m)
        
        flag = 0
        if (len(list1) == 0) :
            if(math.isinf(m)):
                break
            list1.append([m,dis])
        else:
            for i in range(0,len(list1)):
                if ( abs(math.atan(m)-math.atan(list1[i][0])) < 0.5 and abs(dis-list1[i][1])<200):
                    # list1[i][0] = (m+list1[i][0])/2
                    # list1[i][1] = (list1[i][1]+dis)/2
                    flag = 1
                    break
                elif(math.isinf(m)):
                    flag = 1
                    break
                    
            if (flag == 0):               
                list1.append([m,dis])
                     
            
        # cv.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    #cv.imwrite('houghlines3.jpg',img)
    
    # cv.imshow("img",img)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    #shape (640,480 3)
    print(list1)
    print(len(list1))
    nn = len(list1)
    mk = 0
    mr = 0
    lk = 0
    lr = 0
    rk = 0
    rr = 0
    kmax = 1.2
    kmin = -1
    mlmax = 300 #######TODO
    mr_big = 999
    for i in range(0,nn):
        if(abs(list1[i][0])>kmax and list1[i][1] < mr_big):
            mk = list1[i][0]
            mr = list1[i][1]
            mr_big = mr
        if(list1[i][0]<kmax and list1[i][0]>0):
            lk = list1[i][0]
            lr = list1[i][1]
        if(list1[i][0] > kmin and list1[i][0]<0):
            rk = list1[i][0]
            rr = list1[i][1]  
    print("all",[lk,lr,mk,mr,rk,rr])        
    return [lk,lr,mk,mr,rk,rr]




In [35]:
center()

--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Adjusting resolution from 384x288 to 640x480.
--- Capturing frame...
Captured frame in 0.00 seconds.
--- Processing captured image...
Disabling banner.
Writing JPEG image to '/home/xilinx/jupyter_notebooks/common/data/webcam.jpg'.
Writing JPEG image to '2'.
[[-14.238095238095237, 209.66411934819422], [0.8370786516853933, 256.1350715754707], [-0.66666666666666663, 242.40398575042511]]
3
all [0.8370786516853933, 256.1350715754707, -14.238095238095237, 209.66411934819422, -0.66666666666666663, 242.40398575042511]
